In [ ]:
import os
import random
from shutil import copyfile
random.seed(123)
import numpy as np
import math
from unetModel import *

from skimage import io as img_io 

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard

In [ ]:
HOMEDIR = os.getcwd()
BBBCMASK_DIR = '/BBBC_data/mask_creation_images'
# get number of images
ALL_IMAGES = [f for f in os.listdir(HOMEDIR + BBBCMASK_DIR + '/image') if f.endswith('.TIF')]
N_IMAGES = len(ALL_IMAGES)


In [ ]:
ptrain = 0.7
pvalid = 0.1

images_s = random.shuffle(ALL_IMAGES)

ntrain =math.floor(ptrain*N_IMAGES)

train = random.sample(ALL_IMAGES,ntrain)
test = [f for f in ALL_IMAGES if f not in train]

nvalid = (N_IMAGES - ntrain) - math.floor((pvalid/(1-ptrain))*(N_IMAGES - ntrain))

valid = random.sample(test,nvalid)
                                         
test = [f for f in test if f not in valid]

In [ ]:
# fCreate train/valid/test data sets if needed

if not os.path.exists(HOMEDIR + BBBCMASK_DIR + '/train'):
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train/image')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train/image/class0')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train/label')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/train/label/class0')


    
if not os.path.exists(HOMEDIR + BBBCMASK_DIR + '/test'):
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/test')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/test/image')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/test/image/class0')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/test/label')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/test/label/class0')

    
if not os.path.exists(HOMEDIR + BBBCMASK_DIR + '/valid'):
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid/image')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid/image/class0')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid/label')
    os.makedirs(HOMEDIR + BBBCMASK_DIR + '/valid/label/class0')

for f in train:
    raw_image_src = HOMEDIR + BBBCMASK_DIR + '/image/' + f
    raw_image_dest = HOMEDIR + BBBCMASK_DIR + '/train/image/class0/' + f
    mask_image_src = HOMEDIR + BBBCMASK_DIR + '/label/' + f
    mask_image_dest = HOMEDIR + BBBCMASK_DIR + '/train/label/class0/' + f
    copyfile(raw_image_src,raw_image_dest)
    copyfile(mask_image_src,mask_image_dest)

for f in test:
    raw_image_src = HOMEDIR + BBBCMASK_DIR + '/image/' + f
    raw_image_dest = HOMEDIR + BBBCMASK_DIR + '/test/image/class0/' + f
    mask_image_src = HOMEDIR + BBBCMASK_DIR + '/label/' + f
    mask_image_dest = HOMEDIR + BBBCMASK_DIR + '/test/label/class0/' + f
    copyfile(raw_image_src,raw_image_dest)
    copyfile(mask_image_src,mask_image_dest)
                                          
for f in valid:
    raw_image_src = HOMEDIR + BBBCMASK_DIR + '/image/' + f
    raw_image_dest = HOMEDIR + BBBCMASK_DIR + '/valid/image/class0/' + f
    mask_image_src = HOMEDIR + BBBCMASK_DIR + '/label/' + f
    mask_image_dest = HOMEDIR + BBBCMASK_DIR + '/valid/label/class0/' + f
    copyfile(raw_image_src,raw_image_dest)
    copyfile(mask_image_src,mask_image_dest)
    



    


In [ ]:
# Create data generator

data_gen_args = dict(rotation_range = 90,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

def trainGenerator(image_folder,mask_folder,aug_dict,batch_size = 32, image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                   target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        image_folder,
        batch_size = batch_size,
        color_mode = image_color_mode,
        target_size = target_size,
        save_prefix  = image_save_prefix,
        save_to_dir = None,
        class_mode=None,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        mask_folder,
        batch_size = batch_size,
        color_mode = mask_color_mode,
        target_size = target_size,
        save_prefix  = mask_save_prefix,
        save_to_dir= None,
        class_mode=None,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    return(train_generator)




In [ ]:
train_image_dir = HOMEDIR + BBBCMASK_DIR + '/train/image/'
train_mask_dir = HOMEDIR + BBBCMASK_DIR + '/train/label/'
valid_image_dir = HOMEDIR + BBBCMASK_DIR + '/valid/image/'
valid_mask_dir = HOMEDIR + BBBCMASK_DIR + '/valid/label/'
TrainGen = trainGenerator(train_image_dir, train_mask_dir, data_gen_args)
ValidGen = trainGenerator(valid_image_dir, valid_mask_dir, data_gen_args)

In [ ]:
model = unet(input_size = (256,256,1))

In [ ]:
model_checkpoint = ModelCheckpoint('unetCellCnt.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
model_earlyStop = EarlyStopping(monitor='val_loss', min_delta = 0.1, patience = 3)
model_tensorboard = TensorBoard(log_dir = HOMEDIR + '/mask_tb_logs', write_graph=True, write_images=True)


model.fit_generator(TrainGen,steps_per_epoch=ntrain/32,
                        validation_data = ValidGen, 
                        validation_steps = nvalid/32,
                        epochs=50,
                    callbacks=[model_checkpoint, model_earlyStop, model_tensorboard])